In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os
import tqdm
import glob
import tensorflow

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import gray2rgb

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.vgg16 import VGG16
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
import keras.backend as K
from typing import Optional

In [48]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,

        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

In [49]:
train_dataset  = train_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI-CN/MCI-CN/train',
                                                   target_size = (224,224),
                                                   class_mode = 'binary',
                                                   subset = 'training',
                                                   batch_size = 32)

Found 858 images belonging to 2 classes.


In [50]:
test_dataset = test_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI-CN/MCI-CN/test',
                                                  target_size = (224,224),
                                                  class_mode = 'binary',
                                                  batch_size = 32)

Found 286 images belonging to 2 classes.


In [51]:
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')


In [52]:
for layer in base_model.layers:
    layer.trainable=False


In [53]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dense(1, activation='sigmoid'))

In [54]:
def f1_score(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [55]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 5) 

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [56]:

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]


In [57]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=METRICS)

In [58]:
history=model.fit(train_dataset,
                        epochs = 20,
                        verbose = 1,
                         callbacks=lr_scheduler)

Epoch 1/20
27/27 [==============================] - 17s 493ms/step - loss: 0.5417 - accuracy: 0.7249 - precision: 0.7612 - recall: 0.7422 - auc: 0.7977 - f1_score: 0.7524 - lr: 0.0100
Epoch 2/20
27/27 [==============================] - 15s 543ms/step - loss: 0.4061 - accuracy: 0.8298 - precision: 0.8714 - recall: 0.8170 - auc: 0.8913 - f1_score: 0.8398 - lr: 0.0063
Epoch 3/20
27/27 [==============================] - 14s 510ms/step - loss: 0.3603 - accuracy: 0.8613 - precision: 0.8952 - recall: 0.8524 - auc: 0.9139 - f1_score: 0.8707 - lr: 0.0040
Epoch 4/20
27/27 [==============================] - 14s 511ms/step - loss: 0.3100 - accuracy: 0.8811 - precision: 0.9202 - recall: 0.8628 - auc: 0.9357 - f1_score: 0.8894 - lr: 0.0025
Epoch 5/20
27/27 [==============================] - 14s 517ms/step - loss: 0.3496 - accuracy: 0.8590 - precision: 0.8913 - recall: 0.8524 - auc: 0.9197 - f1_score: 0.8706 - lr: 0.0016
Epoch 6/20
27/27 [==============================] - 14s 518ms/step - loss: 0.321

In [59]:
scores = model.evaluate(test_dataset)

9/9 [==============================] - 2s 183ms/step - loss: 0.2134 - accuracy: 0.9231 - precision: 0.9810 - recall: 0.8374 - auc: 0.9784 - f1_score: 0.9005


In [60]:
print("Accuracy = ", scores[1])
print("Precision = ", scores[2])
print("Recall = ", scores[3])
print("AUC = ", scores[4])
print("F1_score = ", scores[5])

Accuracy =  0.9230769276618958
Precision =  0.9809523820877075
Recall =  0.8373983502388
AUC =  0.978352963924408
F1_score =  0.900471031665802
